In [6]:
!pip install tensorflow_hub==0.7
!pip install tensorflow==2.0.0
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 92kB 2.8MB/s 
  Found existing installation: tensorflow-hub 0.8.0
    Uninstalling tensorflow-hub-0.8.0:
      Successfully uninstalled tensorflow-hub-0.8.0
     |████████████████████████████████| 86.3MB 74kB/s 
     |████████████████████████████████| 450kB 45.1MB/s 
     |████████████████████████████████| 3.8MB 47.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=f769b0d39f8ceba2b03d66f7e4f07da8a0a046604c73c1e6fdda5b6de741da26
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: gast 0.3.3
    Uninstall

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.0.0
Hub version:  0.7.0


In [0]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math
import numpy as np
import pandas as pd
from scipy.stats import wasserstein_distance

### Preparing BERT and Functions


In [0]:
max_seq_length = 512  # Your choice here.

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [0]:
# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [0]:
def tokenize_sentence(sentence):
    stokens = tokenizer.tokenize(sentence)
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
    
    input_ids = get_ids(stokens, tokenizer, max_seq_length)
    input_masks = get_masks(stokens, max_seq_length)
    input_segments = get_segments(stokens, max_seq_length)
    
    return input_ids, input_masks, input_segments

def compare_sentences(sentence_1, sentence_2, distance_metric):
    input_ids_1, input_masks_1, input_segments_1 = tokenize_sentence(sentence_1)
    input_ids_2, input_masks_2, input_segments_2 = tokenize_sentence(sentence_2)
    
    pool_embs_1, all_embs_1 = model.predict([[input_ids_1],[input_masks_1],[input_segments_1]])
    pool_embs_2, all_embs_2 = model.predict([[input_ids_2],[input_masks_2],[input_segments_2]])
#     print(pool_embs_1, all_embs_1)
#     print(pool_embs_2, all_embs_2)
    return distance_metric(pool_embs_1[0], pool_embs_2[0])

def distance_between_sentences(sentence_1, sentence_2, distance_metric):
    input_ids_1, input_masks_1, input_segments_1 = tokenize_sentence(sentence_1)
    input_ids_2, input_masks_2, input_segments_2 = tokenize_sentence(sentence_2)
    pool_embs_1, all_embs_1 = model.predict([[input_ids_1],[input_masks_1],[input_segments_1]])
    pool_embs_2, all_embs_2 = model.predict([[input_ids_2],[input_masks_2],[input_segments_2]])
    distances = []
    for i in range(0,max_seq_length):
      distances.append(distance_metric(all_embs_1[0][i], all_embs_2[0][i]))
    distance = np.mean(distances)
    return distance

def get_embs(sentence):
    input_ids, input_masks, input_segments = tokenize_sentence(sentence)
    pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])
    return pool_embs, all_embs

def get_all_embs(sentence):
    print(sentence)
    input_ids, input_masks, input_segments = tokenize_sentence(sentence)
    _, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])
    return all_embs

def square_rooted(x):
    return math.sqrt(sum([a*a for a in x]))

def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return 1-numerator/float(denominator)

def dummy_metric(x,y):
    return 42

## Distances

### Natural Language

In [42]:
s = []
s.append('How are you doing?')
s.append('How much are we feeling?')
s.append('What are you doing?')
s.append('What`s up?')
s.append('Are you doing?')
central = s[0]
print("Central phrase: '{}'".format(central))
for sentence in s:
    print("Cosine Distance to '{}' = {}".format(sentence, round(distance_between_sentences(central, sentence, cosine_similarity), 3)))
    print("Wasserstein Distance to '{}' = {}".format(sentence, round(distance_between_sentences(central, sentence, wasserstein_distance), 3)))
    print("-----------------")

Central phrase: 'How are you doing?'
Cosine Distance to 'How are you doing?' = -0.0
Wasserstein Distance to 'How are you doing?' = 0.0
-----------------
Cosine Distance to 'How much are we feeling?' = 0.292
Wasserstein Distance to 'How much are we feeling?' = 0.066
-----------------
Cosine Distance to 'What are you doing?' = 0.162
Wasserstein Distance to 'What are you doing?' = 0.015
-----------------
Cosine Distance to 'What`s up?' = 0.275
Wasserstein Distance to 'What`s up?' = 0.047
-----------------
Cosine Distance to 'Are you doing?' = 0.244
Wasserstein Distance to 'Are you doing?' = 0.072
-----------------


## Source Code Tokens

In [49]:
w = []
w.append('data')
w.append('Data')
w.append('Data Set')
w.append('datadata')
w.append('dataset')
w.append('DataFrame')
w.append('dataframe')
w.append('df')
w.append('pd.DataFrame')
central = w[6]
print("Central phrase: '{}'".format(central))
for word in w:
    print("Cosine Distance to '{}' = {}".format(word, round(distance_between_sentences(central, word, cosine_similarity), 3)))
    print("Wasserstein Distance to '{}' = {}".format(word, round(distance_between_sentences(central, word, wasserstein_distance), 3)))
    print("-----------------")

Central phrase: 'dataframe'
Cosine Distance to 'data' = 0.214
Wasserstein Distance to 'data' = 0.035
-----------------
Cosine Distance to 'Data' = 0.214
Wasserstein Distance to 'Data' = 0.035
-----------------
Cosine Distance to 'Data Set' = 0.19
Wasserstein Distance to 'Data Set' = 0.013
-----------------
Cosine Distance to 'datadata' = 0.218
Wasserstein Distance to 'datadata' = 0.041
-----------------
Cosine Distance to 'dataset' = 0.141
Wasserstein Distance to 'dataset' = 0.02
-----------------
Cosine Distance to 'DataFrame' = -0.0
Wasserstein Distance to 'DataFrame' = 0.0
-----------------
Cosine Distance to 'dataframe' = -0.0
Wasserstein Distance to 'dataframe' = 0.0
-----------------
Cosine Distance to 'df' = 0.403
Wasserstein Distance to 'df' = 0.073
-----------------
Cosine Distance to 'pd.DataFrame' = 0.186
Wasserstein Distance to 'pd.DataFrame' = 0.071
-----------------


## Source Code Chunks

In [48]:
c = []
cnot = []
c.append("""
def apply_window(image, center, width):
    image = image.copy()

    min_value = center - width // 2
    max_value = center + width // 2

    image[image < min_value] = min_value
    image[image > max_value] = max_value

    return image
""")
c.append("""
def image_windowed(image, custom_center=50, custom_width=130, out_side_val=False):
    '''
    Important thing to note in this function: The image migth be changed in place!
    '''
    # see: https://www.kaggle.com/allunia/rsna-ih-detection-eda-baseline
    min_value = custom_center - (custom_width/2)
    max_value = custom_center + (custom_width/2)
    
    # Including another value for values way outside the range, to (hopefully) make segmentation processes easier. 
    out_value_min = custom_center - custom_width
    out_value_max = custom_center + custom_width
    
    if out_side_val:
        image[np.logical_and(image < min_value, image > out_value_min)] = min_value
        image[np.logical_and(image > max_value, image < out_value_max)] = max_value
        image[image < out_value_min] = out_value_min
        image[image > out_value_max] = out_value_max
    
    else:
        image[image < min_value] = min_value
        image[image > max_value] = max_value
    
    return image
""")
c.append("""
def image_crop(image):
    # Based on this stack overflow post: https://stackoverflow.com/questions/26310873/how-do-i-crop-an-image-on-a-white-background-with-python
    mask = image == 0

    # Find the bounding box of those pixels
    coords = np.array(np.nonzero(~mask))
    top_left = np.min(coords, axis=1)
    bottom_right = np.max(coords, axis=1)

    out = image[top_left[0]:bottom_right[0],
                top_left[1]:bottom_right[1]]
    
    return out
""")
cnot.append("""
def normalize_minmax(img):
    mi, ma = img.min(), img.max()
    return (img - mi) / (ma - mi)""")
cnot.append("""def normalize(img, means, stds, tensor=False):
    return (img - means)/stds""")
cnot.append("""X_train = X_train / 255.0
test = test / 255.0
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)""")
cnot.append("""def standardize(x): 
    return (x-mean_px)/std_px""")
cnot.append("""def rle_decode(mask_rle, shape=(768, 768)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction""")
cnot.append("""def mask2rle(img):
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)""")
cnot.append("""def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img""")
cnot.append("""image = imageio.imread('../input/image1.jpg')
image_rotated = rotate.augment_images([image])
image_noise = gaussian_noise.augment_images([image])
image_crop = crop.augment_images([image])
image_hue = hue.augment_images([image])
image_trans = elastic_trans.augment_images([image])
image_coarse = coarse_drop.augment_images([image])""")

central = c[0]
results_c = []
results_c_wasserstein = []
for chunk in c:
    results_c.append(distance_between_sentences(central, chunk, cosine_similarity))
    results_c_wasserstein.append(distance_between_sentences(central, chunk, wasserstein_distance))

results_cnot = []
results_cnot_wasserstein = []
for chunk in cnot:
    results_cnot.append(distance_between_sentences(central, chunk, cosine_similarity))
    results_cnot_wasserstein.append(distance_between_sentences(central, chunk, wasserstein_distance))
    
print(np.mean(results_c), np.mean(results_cnot))
print(np.mean(results_c_wasserstein), np.mean(results_cnot_wasserstein))

0.28992930316407933 0.34256187616174183
0.06115377509762359 0.05829068909336778


## NL2ML Data

In [0]:
nl2ml = pd.read_csv('/content/nl2ml_images.csv')

In [0]:
cat_names = nl2ml['Preprocessing class (for doc about methods)'].value_counts().head(5).keys().tolist()

In [50]:
central = nl2ml[(nl2ml['Preprocessing class (for doc about methods)'] == cat_names[1])]['Code'].reset_index(drop=True)[0]
results = {"central":cat_names[0]}
for cat in cat_names:
    print(cat)
    rows = nl2ml[(nl2ml['Preprocessing class (for doc about methods)'] == cat)].reset_index(drop=True)
    chunks = rows['Code']
    results_c = []
    for chunk in chunks:
        if len(chunk) <= 512:
            results_c.append(distance_between_sentences(central, chunk, wasserstein_distance))
        else: continue
    results.update({cat:round(np.mean(results_c), 3)})
    del results_c
print(results)

Getting annotations
Visualization
Augmentation
Converting (coordinates)
Resizing
{'central': 'Getting annotations', 'Getting annotations': 0.069, 'Visualization': 0.062, 'Augmentation': 0.077, 'Converting (coordinates)': 0.062, 'Resizing': 0.074}


In [51]:
results

{'Augmentation': 0.077,
 'Converting (coordinates)': 0.062,
 'Getting annotations': 0.069,
 'Resizing': 0.074,
 'Visualization': 0.062,
 'central': 'Getting annotations'}

!!! Very little amount of data used here

## Chunks into Embeddings
Use __get_embs(chunk)__ to get embedding for a chunk

Use __distance_between_sentences(chunk_1, chunk_2, distance_metric)__ to get distance between two chunks

In [0]:
sc_chunk_1 = """
def apply_window(image, center, width):
    image = image.copy()

    min_value = center - width // 2
    max_value = center + width // 2

    image[image < min_value] = min_value
    image[image > max_value] = max_value

    return image
"""
sc_chunk_2 = """
def normalize_minmax(img):
    mi, ma = img.min(), img.max()
    return (img - mi) / (ma - mi)
"""
sc_chunk_3 = """
def lalalalallalalala:
  np.argmax([1,2,3,4,5,6,7,8,9,10])
"""

In [0]:
pool_embs, all_embs = get_embs(sc_chunk_1)

In [55]:
print(distance_between_sentences(sc_chunk_1, sc_chunk_2, cosine_similarity))
print(distance_between_sentences(sc_chunk_1, sc_chunk_2, wasserstein_distance))

0.33366158501913135
0.05372519776344532


In [56]:
print(distance_between_sentences(sc_chunk_2, sc_chunk_3, cosine_similarity))
print(distance_between_sentences(sc_chunk_2, sc_chunk_3, wasserstein_distance))

0.4874605344472024
0.055971320470390606


In [57]:
print(distance_between_sentences(sc_chunk_1, sc_chunk_3, cosine_similarity))
print(distance_between_sentences(sc_chunk_1, sc_chunk_3, wasserstein_distance))

0.5361630921223379
0.06683835241049466


## Getting the Embeddings for the Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
KK_PATH = '/content/drive/My Drive/NL2ML/Tigran_parser/kaggle_kernels/'

In [13]:
df = pd.read_csv(KK_PATH + 'code_blocks.csv', sep='\t')
df.shape

(149170, 2)

In [0]:
df = df.drop_duplicates()

In [0]:
df = df.dropna()

In [0]:
df = df.reset_index(drop=True)

In [0]:
df['code_block_length'] = np.zeros(len(df))
for i in range(len(df)):
  print(str(i)+'\r', end='')
  df['code_block_length'][i] = len(df['code_block'][i])

In [0]:
df = df.drop(df[df['code_block_length'] > 512].index)

In [0]:
df['embeddings'] = np.zeros(df.shape[0])
# df['embeddings'][0:10] = df['code_block'][0:10].apply(get_all_embs) ## Check
df['embeddings'] = df['code_block'].apply(get_all_embs)

#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
#bring in the six packs
#check the decoration
#descriptive statistics summary

#histogram
#skewness and kurtosis
print('Skewness: %f' % df_train['SalePrice'].skew())
#scatter plot grlivarea/saleprice
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
#scatter plot totalbsmtsf/saleprice
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
#box plot overallqual/saleprice
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y='SalePrice', data=data)
var = 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsiz